In [ ]:
!pip install hvplot

In [ ]:
# dependencies .. lets put all the dependencies here
import pandas as pd
from prophet import Prophet
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import os
import numpy as np
import datetime as dt
import hvplot.pandas
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [ ]:
# read the csv in a dataframe
data = pd.read_csv('credit_card_transactions.csv')
# exploration - top 5 records
data.head()

In [ ]:
# Exploration - review data columns
display(data.columns)

print('\n********\n')

# Exploration - unique categories
unique_category = data['category'].unique()
print(unique_category)

print('\n********\n')

# Exploration - unique states
unique_states = data['state'].unique()
print(len(unique_states))
print(unique_states)

In [ ]:
# Exploration - review the datatype
data.info()

In [ ]:
# make a copy to do the transformations
ak_df = data.copy()

# change the 'trans_date_trans_time' column to datetime format
ak_df['trans_date_trans_time'] = pd.to_datetime(ak_df['trans_date_trans_time'])

# change 'dob' to datetime format
ak_df['dob'] = pd.to_datetime(ak_df['dob'])

In [ ]:
# Extract year, month, and day using datetime properties
ak_df['trans_year'] = ak_df['trans_date_trans_time'].dt.year
ak_df['trans_month'] = ak_df['trans_date_trans_time'].dt.month
ak_df['trans_day'] = ak_df['trans_date_trans_time'].dt.day
ak_df.tail()

In [ ]:
# Exploration - review the datatype of updated dataframe
ak_df.info()

In [ ]:
## Cleanup the dataframe
# List of columns to drop
columns_to_drop = [
    'Unnamed: 0', 'first', 'last', 'street', 'lat', 'long', 'merch_zipcode',
    'merch_lat', 'merch_long', 'trans_num', 'unix_time', 'is_fraud', 'zip'
]

# Drop the columns
ak_df_cleaned = ak_df.drop(columns=columns_to_drop)

# Check the remaining columns
ak_df_cleaned.info()

In [ ]:
ak_df_cleaned.head()

In [ ]:
# Categorize time of day based on transaction hour
ak_df_cleaned['time_of_day'] = ak_df_cleaned['trans_date_trans_time'].dt.hour.apply(
    lambda hour: 'Morning' if 6 <= hour < 12 else
    ('Afternoon' if 12 <= hour < 18 else
    ('Evening' if 18 <= hour < 24 else 'Night'))
)

# Check the result
ak_df_cleaned.tail()

In [ ]:
# Calculate age based on the difference between transaction year and birth year
ak_df_cleaned['age'] = ak_df_cleaned['trans_year'] - ak_df_cleaned['dob'].dt.year

ak_df_cleaned.tail()

In [ ]:
# ## test dataframe for an individual CC

# # Filter the DataFrame for the specified credit card number
# filtered_df = ak_df_cleaned[ak_df_cleaned['cc_num'] == 30263540414123]

# # Display the filtered rows
# filtered_df.info()

In [ ]:
# Explore
# Find the minimum and maximum date in the 'trans_date_trans_time' column
date_range = (ak_df_cleaned['trans_date_trans_time'].min(), ak_df_cleaned['trans_date_trans_time'].max())

# Display the range of dates
print(f"The range of transaction dates in the dataset is from {date_range[0].date()} to {date_range[1].date()}.")

print('\n********\n')

# Get the number of unique credit card numbers in the 'cc_num' column
unique_cc_numbers = ak_df_cleaned['cc_num'].nunique()

# Display the result
print(f"The number of unique credit card numbers is: {unique_cc_numbers}")

print('\n********\n')

# Get the count of each gender in the 'gender' column
gender_counts = ak_df_cleaned['gender'].value_counts()

# Display the result
print(gender_counts)

print('\n********\n')

# Group the data by 'gender' and sum the 'amt' for each group
total_transaction_by_gender = ak_df_cleaned.groupby('gender')['amt'].sum()

# Display the result
print(total_transaction_by_gender)

print('\n********\n')

display(ak_df_cleaned['age'].min())
display(ak_df_cleaned['age'].max())

In [ ]:
# Explore - check if there are any null values that needs to be cleaned
# Get the total number of null values in each column
null_values = ak_df_cleaned.isnull().sum()

# Display the total null values
print(null_values)

In [ ]:
# Define the age segmentation function
def age_segment(age):
    if age < 18:
        return 'Under 18'
    elif 19 <= age <= 28:
        return '19-28'
    elif 29 <= age <= 38:
        return '29-38'
    elif 39 <= age <= 48:
        return '39-48'
    elif 49 <= age <= 58:
        return '49-58'
    elif 59 <= age <= 68:
        return '59-68'
    elif 69 <= age <= 78:
        return '69-78'
    else:
        return '79+'

# Apply the function to the 'age' column to create the 'age_group' column
ak_df_cleaned['age_group'] = ak_df_cleaned['age'].apply(age_segment)

# Display the result
ak_df_cleaned.head()

In [ ]:
# Step 1: Group by the 'trans_year' and 'trans_month' and sum the transaction amounts ('amt' column)
monthly_transaction = ak_df_cleaned.groupby(['trans_year', 'trans_month'])['amt'].sum().reset_index()

# Step 2: Combine 'trans_year' and 'trans_month' to create a single label for the x-axis
monthly_transaction['year_month'] = monthly_transaction['trans_year'].astype(str) + '-' + monthly_transaction['trans_month'].astype(str)

# Step 3: Plot the trendline
plt.figure(figsize=(10, 6))
plt.plot(monthly_transaction['year_month'], monthly_transaction['amt'], marker='o')
plt.title('Total Monthly Transactions Over Time')
plt.xlabel('Month-Year')
plt.ylabel('Total Transaction Amount ($)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
## Growth of brand new custoemers

# Create a column that indicates if a customer is making their first transaction
customer_first_transaction = ak_df_cleaned.groupby('cc_num')['trans_date_trans_time'].min().reset_index()
customer_first_transaction['month_year'] = customer_first_transaction['trans_date_trans_time'].dt.to_period('M')

# Count the number of new customers each month
new_customers_per_month = customer_first_transaction.groupby('month_year').size()

# Plot the trend of new customers
plt.figure(figsize=(10, 6))
new_customers_per_month.plot(kind='line', marker='o')
plt.title('Growth of Brand-New Customers Over Time')
plt.xlabel('Month-Year')
plt.ylabel('Number of New Customers')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
## Churn of existing customers

# Step 1: Calculate the last transaction date for each customer
customer_last_transaction = ak_df_cleaned.groupby('cc_num')['trans_date_trans_time'].max().reset_index()

# Step 2: Define churn based on a 90-day threshold
reference_date = pd.to_datetime('2020-06-21')  # Last date in your dataset
customer_last_transaction['days_since_last_transaction'] = (reference_date - customer_last_transaction['trans_date_trans_time']).dt.days

# Step 3: Mark customers as 'Churned' if they have not transacted in the last 90 days
customer_last_transaction['status'] = customer_last_transaction['days_since_last_transaction'].apply(lambda x: 'Churned' if x > 90 else 'Active')

# Step 4: Create a month column from the last transaction date
customer_last_transaction['last_transaction_month'] = customer_last_transaction['trans_date_trans_time'].dt.to_period('M')

# Step 5: Count the number of churned customers each month
churned_customers_over_time = customer_last_transaction[customer_last_transaction['status'] == 'Churned'].groupby('last_transaction_month').size()

# Step 6: Plot the trend of churned customers over time
plt.figure(figsize=(10, 6))
churned_customers_over_time.plot(kind='line', marker='o')
plt.title('Trend of Churned Customers Over Time')
plt.xlabel('Month-Year')
plt.ylabel('Number of Churned Customers')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Group by age group and calculate total spending (amt)
age_group_spending = ak_df_cleaned.groupby('age_group')['amt'].sum()

# Plot the result
plt.figure(figsize=(10, 6))
age_group_spending.plot(kind='bar', color='skyblue')
plt.title('Total Spending per Age Group')
plt.xlabel('Age Group')
plt.ylabel('Total Spending')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# First, create a column for the day of the week (0 = Monday, 6 = Sunday)
ak_df_cleaned['day_of_week'] = ak_df_cleaned['trans_date_trans_time'].dt.dayofweek

# Create a column for weekday/weekend
ak_df_cleaned['day_type'] = ak_df_cleaned['day_of_week'].apply(lambda x: 'Weekday' if x < 5 else 'Weekend')

# Filter for weekdays
weekday_spending = ak_df_cleaned[ak_df_cleaned['day_type'] == 'Weekday'].groupby('age_group')['amt'].sum()

# Plot total spending on weekdays per age group
plt.figure(figsize=(10, 6))
weekday_spending.plot(kind='bar', color='lightgreen')
plt.title('Total Weekday Spending per Age Group')
plt.xlabel('Age Group')
plt.ylabel('Total Weekday Spending')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Filter for weekends
weekend_spending = ak_df_cleaned[ak_df_cleaned['day_type'] == 'Weekend'].groupby('age_group')['amt'].sum()

# Plot total spending on weekends per age group
plt.figure(figsize=(10, 6))
weekend_spending.plot(kind='bar', color='orange')
plt.title('Total Weekend Spending per Age Group')
plt.xlabel('Age Group')
plt.ylabel('Total Weekend Spending')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Summarize:
# Group by 'age_group' and sum the 'amt' column
transaction_amount_by_age_group = ak_df_cleaned.groupby('age_group')['amt'].sum()

# Display the result
print(transaction_amount_by_age_group)

In [ ]:
### CUSTOMER SPENDING SEGMENTATION

# Group by customer (cc_num) and calculate key metrics
customer_data = ak_df_cleaned.groupby('cc_num').agg(
    total_spend=('amt', 'sum'),  # Total amount spent
    transaction_count=('amt', 'count'),  # Number of transactions
    last_transaction=('trans_date_trans_time', 'max')  # Most recent transaction
).reset_index()

# Calculate recency in terms of days since the last transaction
# take reference date as the "last transaction day in dataset + 1"
# customer_data['recency'] = (pd.Timestamp.today() - customer_data['last_transaction']).dt.days

# Get the maximum date in the 'last_transaction' column
reference_date = customer_data['last_transaction'].max() + pd.Timedelta(days=1)

# Calculate recency based on the reference date (last transaction date + 1 day)
customer_data['recency'] = (reference_date - customer_data['last_transaction']).dt.days

# View the customer-level summary
print(customer_data.head())

In [ ]:
# Exploratory code 
customer_data[['cc_num', 'total_spend', 'recency']].sort_values(by='recency', ascending=False).head()

In [ ]:
# Segment customers based on low, moderate, high and top spenders
# Calculate spend thresholds for each quartile
low_spend_threshold = customer_data['total_spend'].quantile(0.25)   # 25th percentile
moderate_spend_threshold = customer_data['total_spend'].quantile(0.50)  # 50th percentile (median)
high_spend_threshold = customer_data['total_spend'].quantile(0.75)  # 75th percentile

# Define a function to classify spenders into four categories based on quartiles
def classify_spender(total_spend):
    if total_spend <= low_spend_threshold:
        return 'Low Spender'         # Bottom 25%
    elif low_spend_threshold < total_spend <= moderate_spend_threshold:
        return 'Moderate Spender'    # 25th to 50th percentile
    elif moderate_spend_threshold < total_spend <= high_spend_threshold:
        return 'High Spender'        # 50th to 75th percentile
    else:
        return 'Top Spender'         # Top 25%

# Apply the function to classify customers into the four segments
customer_data['spending_segment'] = customer_data['total_spend'].apply(classify_spender)

# Display the result
# print(customer_data[['cc_num', 'total_spend', 'spending_segment']].head())
customer_data.head()

In [ ]:
# Define a dictionary to map each spending segment to a specific color
# color mapping was done to ensure that the colors are consistent across both the graphs
color_mapping = {
    'Low Spender': 'red',
    'Moderate Spender': 'orange',
    'High Spender': 'skyblue',
    'Top Spender': 'green'
}

# Calculate the percentage of users in each spending category
user_count_by_category = customer_data['spending_segment'].value_counts(normalize=True) * 100

# Calculate the total spending by each category and the percentage of total spend
spending_by_category = customer_data.groupby('spending_segment')['total_spend'].sum()
spending_percentage_by_category = (spending_by_category / spending_by_category.sum()) * 100

# Use the color mapping to assign the same colors to both charts
colors_user = [color_mapping[category] for category in user_count_by_category.index]
colors_spending = [color_mapping[category] for category in spending_percentage_by_category.index]

# Create a figure with subplots (2 charts side by side)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 7))

# Pie chart for percentage of users by spending category
ax1.pie(user_count_by_category.values, labels=user_count_by_category.index, autopct='%1.1f%%', startangle=140, colors=colors_user)
ax1.set_title('Percentage of Users by Spending Category')

# Pie chart for percentage of total spending by spending category
ax2.pie(spending_percentage_by_category.values, labels=spending_percentage_by_category.index, autopct='%1.1f%%', startangle=140, colors=colors_spending)
ax2.set_title('Percentage of Total Spending by Spending Category')

# Adjust layout to prevent overlap
plt.tight_layout()

# Display the charts
plt.show()

In [ ]:
# Count the number of users in each spending category
user_count_by_category = customer_data['spending_segment'].value_counts(normalize=True) * 100  # Percentage of users
# Calculate the total spending for each category
spending_by_category = customer_data.groupby('spending_segment')['total_spend'].sum()
spending_percentage_by_category = (spending_by_category / spending_by_category.sum()) * 100  # Percentage of total spend

# Create a stacked bar chart
fig, ax = plt.subplots(figsize=(10, 6))

# Plot user percentages (first layer)
ax.bar(user_count_by_category.index, user_count_by_category.values, label='Users %')

# Plot spending percentages (second layer) on top of users
ax.bar(spending_percentage_by_category.index, spending_percentage_by_category.values, bottom=user_count_by_category.values, label='Spending %')

# Add titles and labels
ax.set_title('Percentage of Users and Spending by Category')
ax.set_xlabel('Spending Category')
ax.set_ylabel('Percentage')
plt.legend()
plt.show()

In [ ]:
# Merge the two dataframes on 'cc_num' to combine age groups and spending segments, along with the 'amt' column
merged_df = pd.merge(ak_df_cleaned[['cc_num', 'age_group', 'amt']], customer_data[['cc_num', 'spending_segment']], on='cc_num')
merged_df.head()

In [ ]:
# Group by spending segment and age group to calculate total spending
grouped_data = merged_df.groupby(['spending_segment', 'age_group'])['amt'].sum().unstack()

In [ ]:
# Create a heatmap using hvplot (with no x, y, and C explicitly specified)
heatmap = grouped_data.hvplot.heatmap(
    cmap='coolwarm', 
    colorbar=True, 
    title='Total Spending by Age Group and Spending Segment'
)

# Display the heatmap
heatmap

In [ ]:
# Define quantiles for transaction count
quantiles = customer_data['transaction_count'].quantile([0.25, 0.5, 0.75])

# Classify customers based on transaction count into four segments
def classify_frequency(x):
    if x <= quantiles[0.25]:
        return 'Low Frequency Shopper'
    elif quantiles[0.25] < x <= quantiles[0.5]:
        return 'Moderate Frequency Shopper'
    elif quantiles[0.5] < x <= quantiles[0.75]:
        return 'High Frequency Shopper'
    else:
        return 'Very Frequent Shopper'

# Apply the classification function to create the frequency_segment column
customer_data['frequency_segment'] = customer_data['transaction_count'].apply(classify_frequency)

# Display the updated dataframe
print(customer_data[['cc_num', 'transaction_count', 'frequency_segment']].head(10))

In [ ]:
# Step 1: Define a dictionary to map each frequency segment to a specific color
color_mapping_frequency = {
    'Low Frequency Shopper': 'red',
    'Moderate Frequency Shopper': 'orange',
    'High Frequency Shopper': 'skyblue',
    'Very Frequent Shopper': 'green'
}

# Step 2: Calculate the percentage of users in each frequency category
user_count_by_frequency = customer_data['frequency_segment'].value_counts(normalize=True) * 100

# Step 3: Calculate the total spending by each frequency category and the percentage of total spend
spending_by_frequency = customer_data.groupby('frequency_segment')['total_spend'].sum()
spending_percentage_by_frequency = (spending_by_frequency / spending_by_frequency.sum()) * 100

# Step 4: Use the color mapping to assign the same colors to both charts
colors_user_frequency = [color_mapping_frequency[category] for category in user_count_by_frequency.index]
colors_spending_frequency = [color_mapping_frequency[category] for category in spending_percentage_by_frequency.index]

# Step 5: Create a figure with subplots (2 charts side by side)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 7))

# Pie chart for percentage of users by frequency category
ax1.pie(user_count_by_frequency.values, labels=user_count_by_frequency.index, autopct='%1.1f%%', startangle=140, colors=colors_user_frequency)
ax1.set_title('Percentage of Users by Frequency Segment')

# Pie chart for percentage of total spending by frequency category
ax2.pie(spending_percentage_by_frequency.values, labels=spending_percentage_by_frequency.index, autopct='%1.1f%%', startangle=140, colors=colors_spending_frequency)
ax2.set_title('Percentage of Total Spending by Frequency Segment')

# Adjust layout to prevent overlap
plt.tight_layout()

# Display the charts
plt.show()

In [ ]:
# Group by frequency_segment and calculate the number of unique credit card numbers in each segment
unique_cc_per_frequency = customer_data.groupby('frequency_segment')['cc_num'].nunique()

# Group by frequency_segment and calculate the total transaction amount in each segment
total_spend_per_frequency = customer_data.groupby('frequency_segment')['total_spend'].sum()

# Display the results
print("Unique credit card numbers per frequency segment:")
print(unique_cc_per_frequency)
print("\nTotal transaction amount per frequency segment:")
print(total_spend_per_frequency)

In [ ]:
# Define recency segments based on specific day thresholds
def recency_segment(recency):
    if recency <= 30:
        return 'Very Recent'
    elif 30 < recency <= 60:
        return 'Recent'
    elif 60 < recency <= 120:
        return 'At Risk'
    else:
        return 'Inactive'

# Combine both recency and frequency to create loyalty segments
def combined_loyalty(recency, frequency_segment):
    recency_seg = recency_segment(recency)

    # Combine logic for recency and frequency segments
    if recency_seg == 'Very Recent' and frequency_segment == 'Very Frequent Shopper':
        return 'Very Loyal'
    elif recency_seg == 'Recent' and frequency_segment in ['High Frequency Shopper', 'Very Frequent Shopper']:
        return 'Loyal'
    elif recency_seg == 'At Risk' or frequency_segment == 'Moderate Frequency Shopper':
        return 'At Risk'
    else:
        return 'Inactive'

# Apply the combined loyalty function using both recency and frequency_segment
customer_data['loyalty_segment'] = customer_data.apply(lambda row: combined_loyalty(row['recency'], row['frequency_segment']), axis=1)

# Display the updated dataframe with loyalty segments
customer_data[['cc_num', 'recency', 'transaction_count', 'frequency_segment', 'loyalty_segment']].head(20)


In [ ]:
# Apply the combined loyalty function using both recency and frequency_segment
customer_data['loyalty_segment'] = customer_data.apply(lambda row: combined_loyalty(row['recency'], row['frequency_segment']), axis=1)

# Calculate the total number of users in each loyalty segment
total_users_by_loyalty = customer_data['loyalty_segment'].value_counts()

# Display the result
print(total_users_by_loyalty)

In [ ]:
# Calculate the percentage of users in each loyalty segment
user_count_by_loyalty = customer_data['loyalty_segment'].value_counts(normalize=True) * 100

# Calculate the total spending by each loyalty segment and the percentage of total spending
spending_by_loyalty = customer_data.groupby('loyalty_segment')['total_spend'].sum()
spending_percentage_by_loyalty = (spending_by_loyalty / spending_by_loyalty.sum()) * 100

# Define a dictionary to map each loyalty segment to a specific color
color_mapping_loyalty = {
    'Very Loyal': 'green',
    'Loyal': 'skyblue',
    'At Risk': 'orange',
    'Inactive': 'red'
}

# Use the color mapping to assign the same colors to both charts
colors_loyalty_user = [color_mapping_loyalty[category] for category in user_count_by_loyalty.index]
colors_loyalty_spending = [color_mapping_loyalty[category] for category in spending_percentage_by_loyalty.index]

# Create a figure with subplots (2 charts side by side)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 7))

# Pie chart for percentage of users by loyalty segment
ax1.pie(user_count_by_loyalty.values, labels=user_count_by_loyalty.index, autopct='%1.1f%%', startangle=140, colors=colors_loyalty_user)
ax1.set_title('Percentage of Users by Loyalty Segment')

# Pie chart for percentage of total spending by loyalty segment
ax2.pie(spending_percentage_by_loyalty.values, labels=spending_percentage_by_loyalty.index, autopct='%1.1f%%', startangle=140, colors=colors_loyalty_spending)
ax2.set_title('Percentage of Total Spending by Loyalty Segment')

# Adjust layout to prevent overlap
plt.tight_layout()

# Display the charts
plt.show()

In [ ]:
# Define at-risk customers based on recency (transactions made more than 60 days ago)
at_risk_recency_threshold = 60  # Define at-risk customers as those with transactions older than 60 days

# Classify at-risk customers
customer_data['at_risk_segment'] = customer_data['recency'].apply(lambda x: 'At Risk' if x > at_risk_recency_threshold else 'Not at Risk')

print(customer_data[['cc_num', 'recency', 'at_risk_segment']].head())

In [ ]:
# Combine all segments into one view
customer_data['segment'] = customer_data.apply(
    lambda row: 'High Spender' if row['spending_segment'] == 'High Spender' else
                'Frequent Shopper' if row['frequency_segment'] == 'Frequent Shopper' else
                'Loyal Customer' if row['loyalty_segment'] == 'Loyal Customer' else
                'At Risk', axis=1
)

# Display final segmented data
print(customer_data[['cc_num', 'total_spend', 'transaction_count', 'recency', 'segment']].head())

In [ ]:
# Visualize the customer segmentation
customer_data['segment'].value_counts().plot(kind='bar', figsize=(10,6))
plt.title('Customer Segmentation')
plt.xlabel('Segment')
plt.ylabel('Number of Customers')
plt.show()

In [ ]:
# Calculate total spending for each customer segment
segment_spending = customer_data.groupby('segment')['total_spend'].sum()

# Calculate the percentage of total spend for each segment
segment_spending_percentage = (segment_spending / segment_spending.sum()) * 100

# Display the percentage spending by segment
print(segment_spending_percentage)

In [ ]:
# Create a pie chart of spending percentages by customer segment
plt.figure(figsize=(8, 8))
plt.pie(segment_spending_percentage, labels=segment_spending_percentage.index, autopct='%1.1f%%', startangle=140, colors=['#66b3ff','#99ff99','#ff9999','#ffcc99'])

# Add a title
plt.title('Percentage of Total Spending by Customer Segment')

# Display the pie chart
plt.show()